In [0]:
dbutils.widgets.text("param_data_source", "")
var_data_source = dbutils.widgets.get("param_data_source")

In [0]:
dbutils.widgets.text("param_file_date", "2021-03-21") # based on the name of the subfolder in blob storage
var_file_date = dbutils.widgets.get("param_file_date")

### Read the JSON file using DDL schema

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/utils"

In [0]:
constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
constructor_df = spark.read \
    .schema(constructors_schema) \
    .json(f"{RAW_FOLDER_PATH}/{var_file_date}/constructors.json")



### Drop the unwanted column 

In [0]:
dropped_constructor_df = constructor_df.drop("url")

### Rename and add ingestion date

In [0]:
from pyspark.sql.functions import date_trunc, from_utc_timestamp, current_timestamp, lit

In [0]:
final_constructor_df = add_ingestion_date(dropped_constructor_df.withColumnRenamed("constructorId", "constructor_id") \
                                            .withColumnRenamed("constructorRef", "constructor_ref")) \
                                            .withColumn("data_source", lit(var_data_source)) \
                                            .withColumn("file_date", lit(var_file_date))

display(final_constructor_df)

### Write the parquet file

In [0]:
final_constructor_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.constructors")

In [0]:
%sql 
SELECT COUNT(1) FROM f1_processed.constructors

In [0]:
dbutils.notebook.exit("Success")